In [6]:
import pyomo.environ as pe
import pandas as pd
import numpy as np

In [7]:
solver = pe.SolverFactory('gurobi')

### Decision variables

$x_{ij} = \mathbb{1}\{\text{if node } i \text{ is assigned to node } j \}$

$y_{j} = \mathbb{1}\{\text{if node } j \text{ is a local hub }  \}$

$C_i$: Number of Type C Vans needed for Node i

$A_j$: Number of Type A Vans needed for Hub j
    
### Input Data
$$Dij \quad \quad : Distance \,between\, node\,i\, and \, hub\,j$$
$$gi \quad \quad : Distance \,between\, node\,i\, and \ gateway hub $$
$$di \quad \quad : Customer\, demand\, from\, node\,i $$

### Constraints
 - Node i will only be assigned to node j if j is a local hub.
 - Each node is only assigned to one local hub.
$$
x_{ij} <= y_{j} \ \text{ for all }  i, j \in N \\
\sum_{j \in N} x_{ij} = 1 \ \text{ for each node } i \in N \\
$$

 - Number of vehicles can fulfill the demand.
$$ 40 C_i >= d_i \text{ for each node } i \in N $$
$$ 800 A_j >= \sum_{i \in N} x_{ij} \cdot d_i \text{ for each hub } j \in N$$

### Cost Decomposition
1. Fixed Cost
 - Unit cost: 20
$$F \cdot \sum_{j \in N}y_{j}, F=20$$
2. Cost of Type C Vans
 - Unit cost: 6 * dist
$$ \sum x_{ij} \cdot 6 D_{ij} \cdot C_i$$
3. Cost of Type A Vans 
 - Unit cost: max(70, 70+ 4.5 * (dist - 5))
$$ \sum y_j \cdot max(70, 70+4.5(g_{j}-5)) \cdot A_j$$
$$ g'_{j} = max(5, g_{j})$$
$$ \sum y_j \cdot (70+4.5(g'_{j}-5)) \cdot A_j$$

### Objective Function
$$\text{minimize} \quad 20 \sum_{j \in N}y_{j} + x_{ij} \cdot 6 D_{ij} \cdot C_i + y_j \cdot (70+4.5(g'_{j}-5)) \cdot A_j$$

In [8]:
# import data
demand = pd.read_csv("Data\demand101.csv")
dist = pd.read_csv("Data\dist101.csv")

In [10]:
node = list(demand.customer_code)

In [34]:
mdist = dist.pivot(index="node_1", columns="node_2", values="dist")
mdist

node_2,SF0013,SF0044,SF0065,SF0092,SF0109,SF0207,SF0267,SF0302,SF0305,SF0315,...,SF1878,SF1901,SF1917,SF1923,SF1951,SF1961,SF1965,SF2000,SF2014,SFA
node_1,,,,,,,,,,,,,,,,,,,,,
SF0013,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SF0044,3.027,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SF0065,6.885,6.188,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SF0092,3.561,0.648,6.606,0.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SF0109,12.466,15.325,18.141,15.705,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF1961,20.236,23.260,24.294,23.760,9.341,13.886,7.483,35.282,21.991,8.100,...,29.816,25.762,6.230,6.426,21.967,0.000,NaN,NaN,NaN,NaN
SF1965,8.381,11.321,11.727,11.906,7.730,3.741,19.520,22.724,10.897,18.288,...,17.381,16.895,17.912,11.098,30.829,12.585,0.000,NaN,NaN,NaN
SF2000,12.166,12.760,7.135,13.302,19.799,14.196,28.911,14.458,14.874,26.389,...,13.503,6.842,27.170,23.157,35.869,23.280,12.122,0.000,NaN,34.945


In [35]:
for i in mdist.columns:
    for j in mdist.columns:
        if np.isnan(mdist.loc[i][j]):
            mdist.loc[i][j] = mdist.loc[j][i]

In [36]:
mdist

node_2,SF0013,SF0044,SF0065,SF0092,SF0109,SF0207,SF0267,SF0302,SF0305,SF0315,...,SF1878,SF1901,SF1917,SF1923,SF1951,SF1961,SF1965,SF2000,SF2014,SFA
node_1,,,,,,,,,,,,,,,,,,,,,
SF0013,0.000,3.027,6.885,3.561,12.466,6.408,27.521,16.646,3.038,26.596,...,9.873,18.836,25.993,16.787,39.182,20.236,8.381,12.166,44.266,38.353
SF0044,3.027,0.000,6.188,0.648,15.325,9.410,30.547,14.331,2.827,29.583,...,7.096,19.598,29.015,19.666,42.036,23.260,11.321,12.760,46.918,41.196
SF0065,6.885,6.188,0.000,6.606,18.141,11.791,31.008,10.997,8.790,29.270,...,6.906,13.886,29.338,22.227,40.463,24.294,11.727,7.135,44.349,39.571
SF0092,3.561,0.648,6.606,0.000,15.705,9.887,31.071,14.252,2.786,30.152,...,6.855,20.143,29.550,20.050,42.649,23.760,11.906,13.302,47.557,41.811
SF0109,12.466,15.325,18.141,15.705,0.000,6.358,16.760,28.859,13.481,17.192,...,22.337,24.524,15.571,4.346,30.979,9.341,7.730,19.799,37.716,30.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF1961,20.236,23.260,24.294,23.760,9.341,13.886,7.483,35.282,21.991,8.100,...,29.816,25.762,6.230,6.426,21.967,0.000,12.585,23.280,29.250,21.356
SF1965,8.381,11.321,11.727,11.906,7.730,3.741,19.520,22.724,10.897,18.288,...,17.381,16.895,17.912,11.098,30.829,12.585,0.000,12.122,36.178,30.009
SF2000,12.166,12.760,7.135,13.302,19.799,14.196,28.911,14.458,14.874,26.389,...,13.503,6.842,27.170,23.157,35.869,23.280,12.122,0.000,38.765,34.945


In [38]:
# distance matrix of nodes
ndist = mdist.iloc[:-1,:-1]

# distance with SFA
adist = mdist.iloc[-1]

In [112]:
# g'_j
g = adist.apply(lambda x: max(5, x))

In [113]:
model1 = pe.ConcreteModel()
model1.x = pe.Var(node, node, domain = pe.Binary)
model1.y = pe.Var(node, domain = pe.Binary)
model1.a = pe.Var(node, domain=pe.NonNegativeIntegers)
model1.c = pe.Var(node, domain=pe.NonNegativeIntegers)

In [114]:
model1.costs = pe.Objective(
    expr = 20 * sum(model1.y[j] for j in node) +
    sum(model1.x[i,j] * 6 * ndist.loc[i,j] * model1.c[i] for i in node for j in node) + 
    sum(model1.y[j] * (70+4.5*(g[j]-5)) * model1.a[j] for j in node),
    sense = pe.minimize)

In [115]:
# define 1st set of constraints
def rule_1(mod, i, j):
    return mod.x[i,j] <= mod.y[j]
model1.const1 = pe.Constraint(node, node, rule = rule_1)

def rule_2(mod, i):
    return sum(mod.x[i, j] for j in node) == 1
model1.const2 = pe.Constraint(node, rule = rule_2)

In [116]:
d = demand.set_index(['customer_code']).demand

In [117]:
# define 2nd set of constraints
def rule_3(mod, i):
    return 40 * mod.c[i] >= d[i]
model1.const3 = pe.Constraint(node, rule = rule_3)

def rule_4(mod, j):
    return 800 * mod.a[j] >= sum(mod.x[i,j] * d[i] for i in node)
model1.const4 = pe.Constraint(node, rule = rule_4)

In [118]:
%%time
solver = pe.SolverFactory('gurobi')
result = solver.solve(model1)

Wall time: 44.5 s


In [119]:
model1.costs()

5335.1506803424945

In [120]:
hub = []
for j in node:
    if round(model1.y[j]()) == 1:
        hub.append(j)

In [121]:
hub

['SF0566',
 'SF0747',
 'SF0961',
 'SF1403',
 'SF1801',
 'SF1917',
 'SF1143',
 'SF1365',
 'SF1701',
 'SF0484',
 'SF1267',
 'SF1743',
 'SF1776',
 'SF1245',
 'SF1287']

In [122]:
df = pd.DataFrame(columns = node, index = node)

In [123]:
for i in node:
    for j in node:
        df.loc[i][j] = round(model1.x[i,j]())
df

,SF0566,SF1138,SF0725,SF1781,SF1005,SF1703,SF1398,SF0747,SF0656,SF0841,...,SF1643,SF0302,SF0595,SF1287,SF0904,SF1217,SF0869,SF0557,SF1220,SF0065
SF0566,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF1138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF0725,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
SF1781,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF1005,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF1217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF0869,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF0557,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SF1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
df.to_csv('result1.csv')

In [125]:
for i in node:
    print(model1.c[i]())

2.0
2.0
1.0
2.0
2.0
4.0
2.0
2.0
1.0
2.0
3.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
2.0
3.0
3.0
1.0
2.0
3.0
2.0
1.0
7.0
2.0
3.0
2.0
1.0
1.0
2.0
2.0
3.0
2.0
2.0
2.0
3.0
1.0
1.0
2.0
1.0
1.0
2.0
2.0
2.0
1.0
2.0
3.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
2.0
2.0
1.0
2.0
2.0
3.0
1.0
2.0
1.0
2.0
4.0
2.0
3.0
2.0
3.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
4.0
1.0
1.0
3.0
2.0
2.0
1.0
2.0
1.0
1.0
2.0


In [126]:
for i in hub:
    print(model1.a[i]())

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [127]:
sum(df['SF0566']*d)

555

In [128]:
sum(df['SF0747']*d)

190